Este exemplo é baseado no exemplo já existente do próprio simulador do Netsquid. Entretanto, esse código passou por algumas alterações para fins de melhor compreensão

### Importando todas as bibliotecas necessárias

In [ ]:
import netsquid as ns
from netsquid.examples.entanglenodes import example_network_setup, EntangleNodes
from matplotlib import pyplot as plt
print(f"Este exemplo está localizado em: {ns.examples.entanglenodes.__file__}")

### Criando a classe do simulador

In [ ]:
class Simulation:
    def __init__(self, prep_delay: int = 5, qchannel_delay:int = 100,
                  num_mem_positions: int = 3) -> None:
        self.fidelity = 0.0
        self.network = example_network_setup(prep_delay=prep_delay, qchannel_delay=qchannel_delay,
                                              num_mem_positions=num_mem_positions)
        self.protocol_a = EntangleNodes(node=self.network.subcomponents[
            "node_A"], role="source")
        self.protocol_b = EntangleNodes(node=self.network.subcomponents[
            "node_B"], role="receiver")

    def startSimulation(self) -> None:
        """
        Irá iniciar a simulação desejada
        """
        self.protocol_a.start()
        self.protocol_b.start()
        ns.sim_run()

    def collectFidelity(self) -> None:
        """
        Irá coletar os dados da fidelidade que será analisada
        """
        self.q1, = self.network.subcomponents["node_A"].qmemory.peek(0)
        self.q2, = self.network.subcomponents["node_B"].qmemory.peek(0)
        self.fidelity = ns.qubits.fidelity([self.q1, self.q2], ns.b00)
        print(f"Fidelity of generated entanglement: {self.fidelity}")

    def returnFidelity(self) -> float:
        return self.fidelity

### Criando a classe que irá mostrar os gráficos de fidelidade

In [ ]:
class Graphic:
    def __init__(self, simulation: Simulation, data_file_name: str = 'fidelitydata.txt') -> None:
        self.fidelity_axis = []
        self.data_count = 0
        self.y_label = 'Fidelity'
        self.x_label = 'Execution number'
        self.file_name = data_file_name
        self.fidelity = simulation.returnFidelity()


    def testFile(self, create_file: bool=True) -> bool:
        try:
            with open(self.file_name, 'r') as file:
                return True
            
        except:
            if create_file:
                with open(self.file_name, 'w+'):
                    print(f'Nenhum arquivo encontrado. Criado o arquivo "{self.file_name}"')
            return False
    

    def deleteData(self) -> None:
        test = self.testFile(False)
        if test:
            with open(self.file_name, 'w') as file:
                print("Dados apagados com sucesso")
                return
            
        print("Sem dados para apagar")

    def readDataCollector(self) -> None:
        test = self.testFile()
        if test:
            with open(self.file_name, 'r') as file:
                temp_list = []
                temp_count = 0
                for line in file:
                    if line != None:
                        temp_list.append(float(line.replace('\n', '')))
                        temp_count += 1
                self.fidelity_axis = temp_list
                self.data_count = temp_count
                return
        
        print('Nenhum dado existente')
        return
    
        
    def addOnDataCollector(self) -> None:
        self.testFile()
        with open(self.file_name, '+a') as file:
            file.write(f'{self.fidelity}\n')
            
    
    def printData(self) -> None:
        test = self.testFile()
        if test:
            print("Os dados são: ")
            with open(self.file_name, 'r') as file:
                for line in file:
                    print(line)
                return
        
        print("Nenhum dado pôde ser imprimido")
        return


    def generateGraph(self) -> None:
        plt.plot(self.fidelity_axis, scalex=self.data_count)
        plt.ylabel(self.y_label)
        plt.xlabel(self.x_label)
        plt.show()


### Rodando o simulador

In [ ]:
simulator = Simulation(prep_delay=5, qchannel_delay=100, num_mem_positions=3)
simulator.startSimulation()
simulator.collectFidelity()
graphic = Graphic(simulation=simulator)
graphic.addOnDataCollector()
resp = input("Você gostaria de observar o gráfico dos dados? [s/n] ").split()
if resp[0] in 'Ss':
    graphic.readDataCollector()
    graphic.generateGraph()
    graphic.printData()
    resp = input("Deseja apagar todos os dados salvos? [s/n] ").split()
    if resp[0] in 'sS':
        graphic.deleteData()

else:
    print("Simulação finalizada com sucesso")
